In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
#check if the gpu environment is available
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "none")


In [ ]:
import os
import shutil
from pathlib import Path
import random
import cv2  


# Existing Roboflow dataset path
BFD1_TRAIN_IMG = r"C:\Users\aadit\Downloads\BFD1\train\images"
BFD1_TRAIN_LAB = r"C:\Users\aadit\Downloads\BFD1\train\labels"

BFD1_VAL_IMG   = r"C:\Users\aadit\Downloads\BFD1\valid\images"
BFD1_VAL_LAB   = r"C:\Users\aadit\Downloads\BFD1\valid\labels"

BFD1_TEST_IMG  = r"C:\Users\aadit\Downloads\BFD1\test\images"
BFD1_TEST_LAB  = r"C:\Users\aadit\Downloads\BFD1\test\labels"

# FracAtlas paths
FRAC_ATLAS_FRACTURED_IMG = r"C:\Users\aadit\Downloads\FracAtlas\FracAtlas\images\Fractured"
FRAC_ATLAS_HEALTHY_IMG   = r"C:\Users\aadit\Downloads\FracAtlas\FracAtlas\images\Non_fractured"
FRAC_ATLAS_LABELS        = r"C:\Users\aadit\Downloads\FracAtlas\FracAtlas\Annotations\YOLO"

# path of new dataset
ROOT_COMBINED = r"C:\Final project 2\backend\datasets\Combined_Set"
IMG_TRAIN_DST = os.path.join(ROOT_COMBINED, "images", "train")
IMG_VAL_DST   = os.path.join(ROOT_COMBINED, "images", "valid")
IMG_TEST_DST  = os.path.join(ROOT_COMBINED, "images", "test")

LAB_TRAIN_DST = os.path.join(ROOT_COMBINED, "labels", "train")
LAB_VAL_DST   = os.path.join(ROOT_COMBINED, "labels", "valid")
LAB_TEST_DST  = os.path.join(ROOT_COMBINED, "labels", "test")

for d in [IMG_TRAIN_DST, IMG_VAL_DST, IMG_TEST_DST, LAB_TRAIN_DST, LAB_VAL_DST, LAB_TEST_DST]:
    os.makedirs(d, exist_ok=True)

# histogram enhancement
def enhance_image(src_path, dst_path):
    img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        shutil.copy2(src_path, dst_path)
        return
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(img)
    cv2.imwrite(dst_path, enhanced)

# collect fractured images
fractured_items = []

def collect_bfd1(img_dir, lab_dir, tag):
    out = []
    for f in Path(img_dir).glob("*.*"):
        label_name = f.with_suffix(".txt").name
        lab_path = Path(lab_dir) / label_name
        if lab_path.exists():
            out.append((tag, str(f), str(lab_path)))
    return out

fractured_items += collect_bfd1(BFD1_TRAIN_IMG, BFD1_TRAIN_LAB, "bfd1_train")
fractured_items += collect_bfd1(BFD1_VAL_IMG,   BFD1_VAL_LAB,   "bfd1_valid")
fractured_items += collect_bfd1(BFD1_TEST_IMG,  BFD1_TEST_LAB,  "bfd1_test")

# FracAtlas fractured
for f in Path(FRAC_ATLAS_FRACTURED_IMG).glob("*.*"):
    label_name = f.with_suffix(".txt").name
    label_path = Path(FRAC_ATLAS_LABELS) / label_name
    if label_path.exists():
        fractured_items.append(("fracatlas", str(f), str(label_path)))

print(f"Total fractured images found: {len(fractured_items)}")

# collect healthy images
healthy_items = [str(f) for f in Path(FRAC_ATLAS_HEALTHY_IMG).glob("*.*")]
print(f"Total healthy images found: {len(healthy_items)}")

# balance dataset
n_use = min(len(fractured_items), len(healthy_items))
random.shuffle(fractured_items)
random.shuffle(healthy_items)
fractured_items = fractured_items[:n_use]
healthy_items   = healthy_items[:n_use]
print(f"Using {n_use} fractured and {n_use} healthy images.")

# SPLIT into train/val/test 80/10/10

def split_3(items, train_ratio=0.8, val_ratio=0.1):
    n = len(items)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    train = items[:n_train]
    val = items[n_train:n_train + n_val]
    test = items[n_train + n_val:]
    return train, val, test

frac_train, frac_val, frac_test = split_3(fractured_items, 0.8, 0.1)
heal_train, heal_val, heal_test = split_3(healthy_items, 0.8, 0.1)

# save fractured with labels
def save_fractured(batch, img_dst_root, lab_dst_root, prefix="fracture"):
    for i, (src_type, img_path, lab_path) in enumerate(batch):
        base_name = f"{prefix}_{src_type}_{i:05d}.jpg"
        dst_img_path = os.path.join(img_dst_root, base_name)
        dst_lab_path = os.path.join(lab_dst_root, base_name.replace(".jpg", ".txt"))

        enhance_image(img_path, dst_img_path)
        shutil.copy2(lab_path, dst_lab_path)

# train / val / test for fractured
save_fractured(frac_train, IMG_TRAIN_DST, LAB_TRAIN_DST, "fracture")
save_fractured(frac_val,   IMG_VAL_DST,   LAB_VAL_DST,   "fracture")
save_fractured(frac_test,  IMG_TEST_DST,  LAB_TEST_DST,  "fracture")

# save empty files = healthy
def save_healthy(batch, img_dst_root, lab_dst_root, prefix="healthy"):
    for i, img_path in enumerate(batch):
        base_name = f"{prefix}_{i:05d}.jpg"
        dst_img_path = os.path.join(img_dst_root, base_name)
        dst_lab_path = os.path.join(lab_dst_root, base_name.replace(".jpg", ".txt"))

        enhance_image(img_path, dst_img_path)
        with open(dst_lab_path, "w") as f:
            pass  # empty -> no objects

# train / val / test for healthy
save_healthy(heal_train, IMG_TRAIN_DST, LAB_TRAIN_DST, "healthy")
save_healthy(heal_val,   IMG_VAL_DST,   LAB_VAL_DST,   "healthy")
save_healthy(heal_test,  IMG_TEST_DST,  LAB_TEST_DST,  "healthy")

# write data.yaml
data_yaml_path = os.path.join(ROOT_COMBINED, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(
        "path: C:/Final project 2/backend/datasets/Combined_Set\n"
        "train: images/train\n"
        "val: images/valid\n"
        "test: images/test\n"
        "nc: 2\n"
        "names: ['Fracture', 'Healthy']\n"
    )

print("Combined dataset created at:", ROOT_COMBINED)


In [ ]:
import os
root = r"C:\Final project 2\backend\datasets\Combined_Set"
for s in ["train", "valid", "test"]:
    #count number of image files in each split
    n_img = len(os.listdir(os.path.join(root, "images", s)))
    #count number of label files in each split
    n_lab = len(os.listdir(os.path.join(root, "labels", s)))
    #display counts for verification
    print(f"{s.upper()}: {n_img} images, {n_lab} labels")


In [ ]:
import os
from pathlib import Path
#path for combined dataset
ROOT = Path(r"C:\Final project 2\backend\datasets\Combined_Set")
splits = ["train", "valid", "test"]
#check each splits images one by one
for split in splits:
    #paths for images and labels
    img_dir = ROOT / "images" / split
    lab_dir = ROOT / "labels" / split

    for img_path in img_dir.glob("*.*"):
        #skip unsupported files
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue

        label_path = lab_dir / (img_path.stem + ".txt")

        # if label is empty or missing => treat as healthy
        make_healthy = (not label_path.exists()) or (label_path.stat().st_size == 0)

        if make_healthy:
            # YOLO format: class x_center y_center width height 
            # full-image box for class 1 (Healthy)
            label_path.parent.mkdir(parents=True, exist_ok=True)
            with open(label_path, "w") as f:
                f.write("1 0.5 0.5 1.0 1.0\n")


In [ ]:
from ultralytics import YOLO

def main():
    model = YOLO("yolo11l.pt")  # latest, large

    data_path = r"C:\Final project 2\backend\datasets\Combined_Set\data.yaml"

    model.train(
        data=data_path,# training/val/test dataset
        epochs=350,# number of epochs
        imgsz=640,#image size
        batch=8,#batchg size
        device=0,#gpu index
        project=r"C:\Final project 2\backend\runs_combined",#output directory
        name="yolo11l_unified_2class",#run name
        patience=60,#early stopping
        cache=True,#cache images in ram
        mosaic=1.0,#mosaic augmentation probability
        mixup=0.15,#mixup augmentation probability
        close_mosaic=20,#disable mosaic in last 20 epochs
        fliplr=0.5,#horizontal flip
        flipud=0.0,#vertical flip
        degrees=10.0,#randon rotation
        translate=0.1,#random translation
        scale=0.5,#random scaling
    )

if __name__ == "__main__":
    main()


In [ ]:
from ultralytics import YOLO
#load trained model
model = YOLO(r"C:\Final project 2\backend\runs_combined\yolo11l_unified_2class2\weights\best.pt")
data_path = r"C:\Final project 2\backend\datasets\Combined_Set\data.yaml"

metrics = model.val(data=data_path, split="val")
#print metrices 
print("mAP50:", metrics.box.map50)
print("mAP50-95:", metrics.box.map)
print("Precision:", metrics.box.p.mean())
print("Recall:", metrics.box.r.mean())


In [ ]:
import os
from pathlib import Path
import numpy as np
from ultralytics import YOLO

# paths
img_dir = Path(r"C:\Final project 2\backend\datasets\Combined_Set\images\test")
label_dir = Path(r"C:\Final project 2\backend\datasets\Combined_Set\labels\test")

detector_path = r"C:\Final project 2\backend\runs_combined\yolo11l_unified_2class\weights\best.pt"

# Load detector
detector = YOLO(detector_path)

# helpers
def load_gt_boxes(label_path, img_w, img_h, fracture_class_id=0):
    """Load GT fracture boxes (class 0)."""
    if not label_path.exists() or label_path.stat().st_size == 0:
        return []
    boxes = []
    with open(label_path) as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            if cls != fracture_class_id:
                continue
            xc, yc, w, h = map(float, parts[1:])
            bw, bh = w * img_w, h * img_h
            cx, cy = xc * img_w, yc * img_h
            x1, y1 = cx - bw/2, cy - bh/2
            x2, y2 = cx + bw/2, cy + bh/2
            boxes.append([x1, y1, x2, y2])
    return boxes

def iou(boxA, boxB):
    """Compute IoU for two boxes in xyxy format."""
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    areaA = max(0, boxA[2]-boxA[0]) * max(0, boxA[3]-boxA[1])
    areaB = max(0, boxB[2]-boxB[0]) * max(0, boxB[3]-boxB[1])

    return inter_area / (areaA + areaB - inter_area + 1e-9)


# IoU evaluation
ious = []
count = 0

print("Computing IoU, please wait...")

for img_path in img_dir.glob("*.*"):
    if img_path.suffix.lower() not in ['.jpg', '.png', '.jpeg']:
        continue

    label_path = label_dir / f"{img_path.stem}.txt"

    # GT fracture boxes
    # ofriginal image size will be used
    det_res = detector.predict(str(img_path), conf=0.2, iou=0.55, verbose=False)[0]
    H, W = det_res.orig_shape

    gt_boxes = load_gt_boxes(label_path, W, H, fracture_class_id=0)
    if len(gt_boxes) == 0:
        continue  # skip healthy test images

    # predicted fracture boxes (class 0)
    pred_boxes = [
        box.xyxy[0].tolist()
        for box in det_res.boxes
        if int(box.cls[0]) == 0
    ]

    if len(pred_boxes) == 0:
        # missed detection → IoU = 0 for each GT
        for _ in gt_boxes:
            ious.append(0.0)
        continue

    # best IoU for each GT box
    for gt in gt_boxes:
        best_iou = max(iou(gt, pb) for pb in pred_boxes)
        ious.append(best_iou)

mean_iou = float(np.mean(ious)) if len(ious) else 0.0

print("\n=== Mean IoU Evaluation ===")
print(f"Images Evaluated (fracture only): {len(ious)}")
print(f"Mean IoU: {mean_iou:.3f}")
